In [9]:
import matplotlib.pyplot as plt

In [10]:
import numpy as np
import torch_geometric
import networkx as nx
import os
import torch
import pandas as pd

from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from models.gat_transformer import GATConvTransformer, GATConvLSTM
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from torch_geometric.data import Batch
from models.gcn import GCNConvLSTM

from pathlib import Path

In [11]:
# from typing import Optional, Tuple

# import torch
# from torch import Tensor
# from torch.nn import Parameter, LSTM
# from torch_scatter import scatter_add
# from torch_sparse import SparseTensor, fill_diag, matmul, mul
# from torch_sparse import sum as sparsesum

# from torch_geometric.nn.conv import MessagePassing
# from torch_geometric.nn.dense.linear import Linear
# from torch_geometric.nn.inits import zeros
# from torch_geometric.typing import Adj, OptTensor, PairTensor
# from torch_geometric.utils import add_remaining_self_loops
# from torch_geometric.utils.num_nodes import maybe_num_nodes

In [12]:
#>>> pd.read_excel('data/RhythmNames.xlsx')
#   Acronym Name                                     Full Name
#0          SB                               Sinus Bradycardia
#1          SR                                    Sinus Rhythm
#2         AFIB                            Atrial Fibrillation
#3         ST                                Sinus Tachycardia
#4          AF                                  Atrial Flutter
#5          SI                              Sinus Irregularity
#6          SVT                   Supraventricular Tachycardia
#7          AT                              Atrial Tachycardia
#8         AVNRT  Atrioventricular  Node Reentrant Tachycardia
#9          AVRT        Atrioventricular Reentrant Tachycardia
#10        SAAWR       Sinus Atrium to Atrial Wandering Rhythm

In [13]:
#>>> pd.read_excel('data/ConditionNames.xlsx')
#   Acronym Name                                  Full Name
#0          1AVB            1 degree atrioventricular block
#1          2AVB            2 degree atrioventricular block
#2         2AVB1  2 degree atrioventricular block(Type one)
#3         2AVB2  2 degree atrioventricular block(Type two)
#4          3AVB            3 degree atrioventricular block
#5           ABI                            atrial bigeminy
#6           ALS                            Axis left shift
#7           APB                     atrial premature beats
#8           AQW                            abnormal Q wave
#9           ARS                           Axis right shift
#10          AVB                     atrioventricular block
#11          CCR                 countercolockwise rotation
#12           CR                        colockwise rotation
#13          ERV     Early repolarization of the ventricles
#14         FQRS                                  fQRS Wave
#15          IDC            Interior differences conduction
#16          IVB                     Intraventricular block
#17          JEB                     junctional escape beat
#18          JPS                              J point shift
#19          JPT                  junctional premature beat
#20         LBBB                   left bundle branch block
#21        LBBBB              left back bundle branch block
#22        LFBBB             left front bundle branch block
#23         LRRI                           Long RR interval
#24          LVH                 left ventricle hypertrophy
#25         LVHV                left ventricle high voltage
#26      LVQRSAL              lower voltage QRS in all lead
#27      LVQRSCL            lower voltage QRS in chest lead
#28      LVQRSLL             lower voltage QRS in limb lead
#29           MI                      myocardial infarction
#30         MIBW         myocardial infraction in back wall
#31         MIFW   Myocardial infgraction in the front wall
#32         MILW    Myocardial infraction in the lower wall
#33         MISW     Myocardial infraction in the side wall
#34         PRIE                      PR interval extension
#35          PWC                              P wave Change
#36         QTIE                      QT interval extension
#37          RAH                   right atrial hypertrophy
#38         RAHV                  right atrial high voltage
#39         RBBB                  right bundle branch block
#40          RVH                right ventricle hypertrophy
#41         STDD                               ST drop down
#42          STE                               ST extension
#43         STTC                                ST-T Change
#44         STTU                                 ST tilt up
#45          TWC                              T wave Change
#46          TWO                            T wave opposite
#47           UW                                     U wave
#48           VB                       ventricular bigeminy
#49          VEB                    ventricular escape beat
#50          VFW                    ventricular fusion wave
#51          VPB                 ventricular premature beat
#52          VPE                  ventricular preexcitation
#53          VET               ventricular escape trigeminy
#54         WAVN   Wandering in the atrioventricalualr node
#55          WPW                                        WPW

In [14]:
def get_edge_index(node_num=12):
    src_nodes = [idx for idx in range(node_num) for _ in range(node_num)]
    tgt_nodes = [idx for _ in range(node_num) for idx in range(node_num)]
    edge_index = torch.tensor(np.array([src_nodes, tgt_nodes]), dtype=torch.long)
    return edge_index

In [15]:
diagnostics_df = pd.read_excel('data/Diagnostics.xlsx')

In [16]:
le = LabelEncoder()
le.fit(diagnostics_df.Rhythm.unique())

LabelEncoder()

In [17]:
le.classes_

array(['AF', 'AFIB', 'AT', 'AVNRT', 'AVRT', 'SA', 'SAAWR', 'SB', 'SR',
       'ST', 'SVT'], dtype=object)

In [18]:
diagnostics_df['label'] = le.transform(diagnostics_df.Rhythm.values)

In [19]:
diagnostics_df['label']

0         1
1         7
2         5
3         7
4         0
         ..
10641    10
10642    10
10643    10
10644    10
10645    10
Name: label, Length: 10646, dtype: int64

In [20]:
data_list = []
data_path = Path('data', 'ECGDataDenoised')
for file_name in tqdm(data_path.iterdir()):
    if file_name.suffix == '.csv':
        sub_name = file_name.stem
        y = torch.tensor(diagnostics_df.loc[diagnostics_df.FileName == sub_name].label.values[0])
        df = pd.read_csv(file_name, header=None)
        if len(df) != 5000:
            print(file_name)
            continue
        x = torch.Tensor(np.array([df[col].values for col in df.columns]))
        edge_index = get_edge_index(node_num=12)
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)

5760it [00:50, 116.02it/s]

data/ECGDataDenoised/MUSE_20180113_124215_52000.csv


10646it [01:32, 114.84it/s]


In [34]:
train_loader = DataLoader(data_list[:8000], batch_size=8, shuffle=True)
test_loader = DataLoader(data_list[8000:], batch_size=8)

In [22]:
instance = next(iter(train_loader))

In [23]:
instance

DataBatch(x=[192, 5000], edge_index=[2, 2304], y=[16], batch=[192], ptr=[17])

In [24]:
#g = torch_geometric.utils.to_networkx(instance, to_undirected=True)
#nx.draw(g)

In [25]:
instance.y[0]

tensor(7)

In [26]:
hidden_channels = 250
device = 'cpu' if not torch.cuda.is_available() else 'cuda'

In [62]:
# Training Graph Neural Networks for Graph Classification
# Embed each node by performing multiple rounds of message passing
# Aggregate node embeddings into a unified graph embedding (readout layer)
# Train a final classifier on the graph embedding
 
# There exists multiple readout layers in literature, but the most common one is to simply take the average of node embeddings

# For ex: In GCNConv we use the  ReLU(𝑥)=max(𝑥,0)  activation for obtaining localized node embeddings,
# before we apply our final classifier on top of a graph readout layer.

# PyTorch Geometric provides this functionality via torch_geometric.nn.global_mean_pool, 
# which takes in the node embeddings of all nodes in the mini-batch and the assignment vector batch 
# to compute a graph embedding of size [batch_size, hidden_channels] for each graph in the batch.

# The final architecture for applying GNNs to the task of graph classification then looks as follows and allows for complete end-to-end training:

from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import GraphConv


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        self.conv1 = GraphConv(-1, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(-1, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x


class GAT(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GAT, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(-1, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
    
class GCNLSTM(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCNLSTM, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConvLSTM(1, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, len(le.classes_))

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x
       
# Define the models
model1 = GCN(hidden_channels=hidden_channels).to(device)
model2= GAT(hidden_channels=hidden_channels).to(device)
model3 = GNN(hidden_channels=hidden_channels).to(device)
model4 = GCNLSTM(hidden_channels=hidden_channels).to(device)

# Print them 
print(model1)
print(model2)
print(model3)
print(model4)

GCN(
  (conv1): GCNConv(-1, 250)
  (bn1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNConv(250, 250)
  (bn2): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GCNConv(250, 250)
  (lin): Linear(in_features=250, out_features=11, bias=True)
)
GAT(
  (conv1): GATConv(-1, 250, heads=1)
  (bn1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GATConv(250, 250, heads=1)
  (bn2): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): GATConv(250, 250, heads=1)
  (lin): Linear(in_features=250, out_features=11, bias=True)
)
GNN(
  (conv1): GraphConv(-1, 250)
  (conv2): GraphConv(250, 250)
  (conv3): GraphConv(250, 250)
  (lin): Linear(in_features=250, out_features=11, bias=True)
)
GATLSTM(
  (conv1): GCNConvLSTM(1, 250)
  (bn1): BatchNorm1d(250, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): GCNC

In [63]:
# Set model paramters and model type
def set_model_parameters(model_type, lr=0.01):
    model = model_type
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    return model, optimizer, criterion

# Train the model
def train(model, optimizer,criterion):
    model.train()

    for idx, data in enumerate(train_loader):  # Iterate in batches over the training dataset.
        data.x = data.x.to(device)
        data.y = data.y.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        
        if idx % 500 == 0:
            print(f'{idx}tr) loss: {loss}')
    return

# Test the model 
def test(loader, model):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        data.x = data.x.to(device)
        data.y = data.y.to(device)
        data.edge_index = data.edge_index.to(device)
        data.batch = data.batch.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

# Training and Testing Pipeline 
def running_epochs(model,optimizer,criterion):
    for epoch in range(1, 10):
        train(model,optimizer,criterion)
        train_acc = test(train_loader, model)
        test_acc = test(test_loader, model)
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [64]:
# Experiment 1: GCN Baseline 
model, optimizer, criterion = set_model_parameters(model1, lr=0.001)
running_epochs(model,optimizer,criterion)

0tr) loss: 2.8912720680236816
500tr) loss: nan


KeyboardInterrupt: 

In [44]:
# Experiment 4: GCN Baseline 
model, optimizer, criterion = set_model_parameters(model4, lr=0.001)
running_epochs(model,optimizer,criterion)

output shape: torch.Size([8, 11])
y shape: torch.Size([8])
0tr) loss: 2.3035831451416016
Epoch: 001, Train Acc: 0.0408, Test Acc: 0.0450
output shape: torch.Size([8, 11])
y shape: torch.Size([8])
0tr) loss: nan
Epoch: 002, Train Acc: 0.0408, Test Acc: 0.0450
output shape: torch.Size([8, 11])
y shape: torch.Size([8])
0tr) loss: nan


KeyboardInterrupt: 